In [ ]:
import numpy as np
import scipy.signal
import scipy.ndimage
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

def asymptotic_update_demo():
    size = 64
    mid = size // 2
    R = 13
    T = 10
    mu = 0.15
    sigma = 0.015
    
    # Initialize random world centered
    A = np.zeros([size, size])
    noise = np.random.rand(20, 20)
    A[mid-10:mid+10, mid-10:mid+10] = noise

    # Bell function
    bell = lambda x, m, s: np.exp(-((x-m)/s)**2 / 2)

    # --- CORRECCIÓN AQUÍ ---
    # En lugar de np.linalg.norm, usamos broadcasting manual
    Y, X = np.ogrid[-mid:mid, -mid:mid] 
    D = np.sqrt(X**2 + Y**2) / R
    
    K = (D < 1) * bell(D, 0.5, 0.15)
    fK = np.fft.fft2(np.fft.fftshift(K / np.sum(K)))

    # DEFINING THE TARGET FUNCTION
    def target_func(U):
        return bell(U, mu, sigma)

    def update(i):
        nonlocal A
        # Convolution via FFT
        U = np.real(np.fft.ifft2(fK * np.fft.fft2(A)))
        
        # Asymptotic Update: 
        # New_State = Current + (1/T) * (Target - Current)
        A = np.clip(A + 1/T * (target_func(U) - A), 0, 1)
        
        img.set_array(A)
        return img,

    fig = plt.figure(figsize=(5,5))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.axis('off')
    # Usamos 'magma' para mejor contraste
    img = ax.imshow(A, cmap='magma', vmin=0, vmax=1)
    
    return animation.FuncAnimation(fig, update, frames=200, interval=20, blit=True)

# Run the animation
anim = asymptotic_update_demo()
anim.save("asymptotic_update_demo.mp4", writer="ffmpeg", fps=30)
HTML(anim.to_jshtml())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

def mixed_physics_demo():
    size = 128
    mid = size // 2
    R = 12
    dt = 0.1
    
    # --- 1. INICIALIZACIÓN (3 CANALES: R, G, B) ---
    # Creamos 3 mundos separados apilados
    # Shape: (3, 128, 128)
    A = np.zeros((3, size, size))
    
    # Ponemos ruido aleatorio en el centro para arrancar
    noise = np.random.rand(3, 40, 40)
    A[:, mid-20:mid+20, mid-20:mid+20] = noise

    # --- 2. PRE-CÁLCULO DEL KERNEL ---
    # Usamos el mismo kernel para todos para simplificar la demo
    Y, X = np.ogrid[-mid:mid, -mid:mid]
    D = np.sqrt(X**2 + Y**2) / R
    
    bell = lambda x, m, s: np.exp(-((x-m)/s)**2 / 2)
    K = (D < 1) * bell(D, 0.5, 0.15)
    
    # Normalizamos y pasamos a Fourier (FFT)
    K_sum = np.sum(K)
    fK = np.fft.fft2(np.fft.fftshift(K / K_sum))

    # --- 3. DEFINICIÓN DE FÍSICAS ---
    
    # Función de Crecimiento (Lenia Clásica): Explosiva, biológica
    def growth_func(U, m, s):
        return bell(U, m, s) * 2 - 1

    # Función de Objetivo (Asymptotic): Suave, fantasmagórica
    # La célula intenta igualar el valor de la activación U
    def target_func(U, m, s, current_val):
        return bell(U, m, s) - current_val

    # --- 4. BUCLE DE ACTUALIZACIÓN ---
    def update(frame):
        nonlocal A
        
        # Transformamos los 3 canales a frecuencia
        fA = np.fft.fft2(A, axes=(1, 2))
        
        # Calculamos la percepción (Convolución) para cada canal
        # U tendrá shape (3, 128, 128)
        U = np.real(np.fft.ifft2(fK * fA, axes=(1, 2)))
        
        # --- APLICAMOS FÍSICAS MIXTAS ---
        
        # Canal 0 (ROJO): Física de Crecimiento (Agresiva)
        # Mu=0.15, Sigma=0.015 (Gusano estándar)
        delta_r = growth_func(U[0], 0.15, 0.015)
        
        # Canal 1 (VERDE): Física de Crecimiento (Un poco distinta)
        # Mu=0.14 (Le gusta estar más solo)
        delta_g = growth_func(U[1], 0.14, 0.015)
        
        # Canal 2 (AZUL): Física de OBJETIVO (Target/Asymptotic)
        # Este canal intenta suavemente acercarse al estado del canal Rojo (U[0])
        # Esto crea un efecto de "estela" o memoria.
        delta_b = target_func(U[0], 0.15, 0.015, A[2])

        # Actualizamos el estado
        # R y G se alimentan de sí mismos, B reacciona a R
        A[0] = np.clip(A[0] + dt * delta_r, 0, 1)
        A[1] = np.clip(A[1] + dt * delta_g, 0, 1)
        A[2] = np.clip(A[2] + dt * delta_b, 0, 1) # B es suave
        
        # Preparamos imagen para matplotlib (Transpose a H, W, Channels)
        img_data = A.transpose(1, 2, 0)
        img.set_array(img_data)
        return img,

    # --- RENDERIZADO ---
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.axis('off')
    
    # Estado inicial visual
    img_data = A.transpose(1, 2, 0)
    img = ax.imshow(img_data, vmin=0, vmax=1)
    
    return animation.FuncAnimation(fig, update, frames=200, interval=30, blit=True)

anim = mixed_physics_demo()
anim.save("mixed_physics_demo.mp4", writer="ffmpeg", fps=30)
HTML(anim.to_jshtml())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

def soft_clip_demo():
    size = 64
    mid = size // 2
    R = 12
    dt = 0.1
    
    # Inicialización con ruido aleatorio
    A = np.random.rand(size, size)
    
    # Función de Campana (Gaussian)
    bell = lambda x, m, s: np.exp(-((x-m)/s)**2 / 2)
    
    # --- PREPARAR KERNEL (FFT) ---
    Y, X = np.ogrid[-mid:mid, -mid:mid]
    D = np.sqrt(X**2 + Y**2) / R
    
    # Kernel de anillo suave
    K = (D < 1) * bell(D, 0.5, 0.15)
    fK = np.fft.fft2(np.fft.fftshift(K / np.sum(K)))

    # --- LA MAGIA: SOFT CLIP (SIGMOIDE) ---
    # En lugar de cortar bruscamente, aplastamos los valores suavemente.
    # La fórmula 1 / (1 + exp(-4 * (x - 0.5))) centra la curva en 0.5
    def soft_clip(x):
        return 1 / (1 + np.exp(-4 * (x - 0.5)))

    def update(i):
        nonlocal A
        
        # 1. Percepción (Convolución FFT)
        fA = np.fft.fft2(A)
        U = np.real(np.fft.ifft2(fK * fA))
        
        # 2. Crecimiento (Lenia Clásica)
        # Mu=0.15, Sigma=0.015 (Parámetros estándar de vida)
        growth = bell(U, 0.15, 0.015) * 2 - 1
        
        # 3. Actualización Potencial
        A_temp = A + dt * growth
        
        # 4. APLICAR SOFT CLIP
        # Aquí es donde difiere del Lenia original.
        # No usamos np.clip(A_temp, 0, 1)
        A = soft_clip(A_temp)
        
        img.set_array(A)
        return img,

    # --- RENDERIZADO ---
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.axis('off')
    
    # Usamos 'viridis' para ver los gradientes suaves que produce la sigmoide
    img = ax.imshow(A, cmap='viridis', vmin=0, vmax=1)
    
    return animation.FuncAnimation(fig, update, frames=200, interval=20, blit=True)

anim = soft_clip_demo()
anim.save("soft_clip_demo.mp4", writer="ffmpeg", fps=30)
HTML(anim.to_jshtml())